In [115]:
# Import Necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read Data
ev_data = pd.read_csv('../Data/IEA-EV-data.csv')

In [3]:
ev_df = (
ev_data
    .assign(
        **{c:lambda df_,c=c:df_[c].astype('category') for c in ['region','category','parameter','mode','powertrain','unit']},
        year = lambda df_:df_['year'].astype("int16")
    )
)

## Get Charging Points Related Data by Filters

In [4]:
# Get only historical data of charging points.
ev_his_df = ev_df[(ev_df["category"]=="Historical") & (ev_df["parameter"] == "EV charging points")]
# ev_his_df = ev_df.query("category=='Historical' and parameter == 'EV charging points'")
ev_his_df.head()

,region,category,parameter,mode,powertrain,year,unit,value
38,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.000000
39,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,436.000000
43,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.093887
44,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,665.923340
55,Australia,Historical,EV charging points,EV,Publicly available fast,2019,charging points,251.000000


In [5]:
# ev_his_df.category.value_counts()
# ev_his_df.parameter.value_counts()

In [130]:
def _get_european_countries(df_:pd.DataFrame)->pd.DataFrame:
    EUROPEAN_COUNTRIES = ['Europe','Other Europe','Russia','Germany', 
                          'United Kingdom', 'France', 'Italy', 'Spain', 'Ukraine',
                           'Poland', 'Romania', 'Netherlands', 'Belgium',
                           'Czech Republic (Czechia)', 'Greece', 'Portugal', 'Sweden',
                           'Hungary', 'Belarus', 'Austria', 'Serbia', 'Switzerland',
                           'Bulgaria', 'Denmark', 'Finland', 'Slovakia', 'Norway', 'Ireland',
                           'Croatia', 'Moldova', 'Bosnia and Herzegovina', 'Albania',
                           'Lithuania', 'North Macedonia', 'Slovenia', 'Latvia', 'Estonia',
                           'Montenegro', 'Luxembourg', 'Malta', 'Iceland', 'Andorra',
                           'Monaco', 'Liechtenstein', 'San Marino', 'Holy See']
    
    conditions = [
       df_['region'].isin(EUROPEAN_COUNTRIES),
    ]

    choices = [
        'Europe'
    ]

    df_['region_class'] = np.select(conditions, choices, default=df_['region'].values)
    return df_

In [131]:
from typing import List
def get_charger_type_count_by_country(df_:pd.DataFrame, countries: List[str], charger_type:str)->pd.DataFrame:
    return (df_
            .pipe(_get_european_countries)
            .query(f"region_class in {countries} and powertrain=='Publicly available {charger_type}'")
            .assign(
#                     region     = lambda df_:df_["region"].to_list(),
                    powertrain = lambda df_:df_["powertrain"].to_list(),
            )
            .groupby(['region_class','year','powertrain'])['value']
            .sum()
            .reset_index()
            .assign(
                value= lambda df_:df_["value"]/1000
            )
            .sort_values("year")
            .reset_index(drop = True)
           )

### Fast Charging stations in `['China','Europe','USA','India']` by year.

In [132]:
data_fast = get_charger_type_count_by_country(ev_his_df,['China','India','USA','Europe'],'fast')

In [133]:
import plotly.express as px
fig = px.bar(data_fast, x='year', y='value',color="region_class",
             title="Fast publicly available chargers, 2014-2021",
             labels=dict(year="", value="Fast Charger Public Stock (Thousands)", region_class="Countries"))
fig.show()

### Slow Charging stations in `['China','Europe','USA','India']` by year.

In [121]:
data_slow = get_charger_type_count_by_country(ev_his_df,['China','India','USA','Europe'],'slow')

In [122]:
fig = px.bar(data_slow, x='year', y='value',color="region_class",
             title="Slow publicly available chargers, 2010-2021",
            labels=dict(year="", value="Slow Charger Public Stock (Thousands)", region_class="Countries"))
fig.show()

### Charging Ports Per EV

In [252]:
ev_stock = (ev_data
    .query("region not in ['Europe','Other Europe','World','Rest of the world']\
        and unit=='stock' and category=='Historical' and year==2021")
    .groupby("region")['value']
    .sum()
    .reset_index()
    .sort_values("value",ascending=False)
    .set_index("region")
)

In [246]:
ev_charging_points = (ev_data
    .query("region not in ['Europe','Other Europe','World','Rest of the world']\
           and unit=='charging points' and category=='Historical' and year=='2021'")
    .groupby("region")['value']
    .sum()
    .reset_index()
    .sort_values("value",ascending=False)
    .set_index("region")
)

In [226]:
merged_df = ev_stock.merge(ev_charging_points,left_index=True,right_index=True,suffixes=("_stock","_charging"))

In [237]:
ev_per_cp = (merged_df
                 .assign(
                    ev_per_charging_points = lambda df_: round(df_['value_stock']/df_['value_charging'],2)
                )
                 .sort_values("ev_per_charging_points", ascending=False)
            )

In [244]:
fig = px.bar(
            ev_per_cp,
            x= ev_per_cp["ev_per_charging_points"],
            y=ev_per_cp.index.to_list(),
            orientation='h',
            title="Charging points per EV in selected countries, 2021",
            labels=dict(y="Countries", ev_per_charging_points="ELectric Car Per Charging Point")
            )
fig.show()